In [80]:
import pandas as pd

In [81]:
survey_df=pd.read_csv('cleaned_survey_data.csv')
survey_df.head()

,respondent_id,age,gender,zone,occupation,income_levels,consume_frequency(weekly),current_brand,preferable_consumption_size,awareness_of_other_brands,reasons_for_choosing_brands,flavor_preference,purchase_channel,packaging_preference,health_concerns,typical_consumption_situations,price_range
0,R00001,30,M,Urban,Working Professional,<10L,3-4 times,Newcomer,Medium (500 ml),0 to 1,Price,Traditional,Online,Simple,Medium (Moderately health-conscious),"Active (eg. Sports, gym)",100-150
1,R00002,46,F,Metro,Working Professional,> 35L,5-7 times,Established,Medium (500 ml),2 to 4,Quality,Exotic,Retail Store,Premium,Medium (Moderately health-conscious),Social (eg. Parties),200-250
2,R00003,41,F,Rural,Working Professional,> 35L,3-4 times,Newcomer,Medium (500 ml),2 to 4,Availability,Traditional,Retail Store,Premium,Medium (Moderately health-conscious),"Active (eg. Sports, gym)",200-250
3,R00004,33,F,Urban,Working Professional,16L - 25L,5-7 times,Newcomer,Medium (500 ml),0 to 1,Brand Reputation,Exotic,Online,Eco-Friendly,Low (Not very concerned),"Active (eg. Sports, gym)",150-200
4,R00005,23,M,Metro,Student,Not Reported,3-4 times,Established,Medium (500 ml),0 to 1,Availability,Traditional,Online,Premium,Medium (Moderately health-conscious),"Active (eg. Sports, gym)",50-100


survey_df = survey_df.astype({col: 'category'  if col!='respondent_id' else 'object' for col in survey_df.select_dtypes(include='object').columns})


In [82]:
survey_df.dtypes

respondent_id                     object
age                                int64
gender                            object
zone                              object
occupation                        object
income_levels                     object
consume_frequency(weekly)         object
current_brand                     object
preferable_consumption_size       object
awareness_of_other_brands         object
reasons_for_choosing_brands       object
flavor_preference                 object
purchase_channel                  object
packaging_preference              object
health_concerns                   object
typical_consumption_situations    object
price_range                       object
dtype: object

# Step 1: Categorize Age into Age Groups
 Objective: Create a new column `age_group` by categorizing the existing `age`
 
 column into specific age ranges.
 - Categorize ages into the following groups:
     - 18-25
     - 26-35
     - 36-45
     - 46-55
     - 56-70
     - 70+
       
 - Ensure that each age in the dataset is mapped to one of these age groups.
 - After that, remove the original `age` column from the dataset.

In [83]:
survey_df_2=survey_df.copy()


### We can use .apply() also but  i am going with .cut() : Method i find by chatgpt

In [84]:
# why we write 17??? because in cut(right=True) means right value is included but not left 
# so if we write 18 then it will not be included in 18-25
age_bins = [17, 25, 35, 45, 55, 70, float('inf')]  # Define bins
age_labels = ['18-25', '26-35', '36-45', '46-55', '56-70', '70+']  # Define labels

survey_df_2['age_group'] = pd.cut(survey_df_2['age'], bins=age_bins, labels=age_labels, right=True)

In [85]:
survey_df_2=survey_df_2.drop('age',axis=1)

In [86]:
survey_df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29991 entries, 0 to 29990
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   respondent_id                   29991 non-null  object  
 1   gender                          29991 non-null  object  
 2   zone                            29991 non-null  object  
 3   occupation                      29991 non-null  object  
 4   income_levels                   29991 non-null  object  
 5   consume_frequency(weekly)       29991 non-null  object  
 6   current_brand                   29991 non-null  object  
 7   preferable_consumption_size     29991 non-null  object  
 8   awareness_of_other_brands       29991 non-null  object  
 9   reasons_for_choosing_brands     29991 non-null  object  
 10  flavor_preference               29991 non-null  object  
 11  purchase_channel                29991 non-null  object  
 12  packaging_preferen

# Step 2: Create `cf_ab_score` (Consume Frequency and Awareness Brand Score)
 
 Objective: Create a new column `cf_ab_score` by combining the information from
 `consume_frequency(weekly)` and `awareness_of_other_brands` columns.
 - Usethefollowing mappings for consume_frequency(weekly):
     - "0-2 times" → 1
     - "3-4 times" → 2
     - "5-7 times" → 3
 - Usethefollowing mappings for awareness_of_other_brands:
     - "0to1" →1
     - "2to4" →2
     - "above 4" →3
 - Calculate cf_ab_score using the following formula:
  $$
\text{cf\_ab\_score} = \frac{\text{frequency\_score}}{\text{awareness\_score} + \text{frequency\_score}}
$$

 - Roundthe result to two decimal places

In [87]:
survey_df_3=survey_df_2.copy()

In [88]:
survey_df_3['consume_frequency(weekly)'].unique()

array(['3-4 times', '5-7 times', '0-2 times'], dtype=object)

In [89]:
survey_df_3['awareness_of_other_brands'].unique()

array(['0 to 1', '2 to 4', 'above 4'], dtype=object)

In [90]:

consume_frequency_mapping = {
    "0-2 times": 1,
    "3-4 times": 2,
    "5-7 times": 3
}

awareness_mapping = {
    "0 to 1": 1,
    "2 to 4": 2,
    "above 4": 3
}

survey_df_3['frequency_score'] = survey_df_3['consume_frequency(weekly)'].map(consume_frequency_mapping)
survey_df_3['awareness_score'] = survey_df_3['awareness_of_other_brands'].map(awareness_mapping)




In [91]:
survey_df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29991 entries, 0 to 29990
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   respondent_id                   29991 non-null  object  
 1   gender                          29991 non-null  object  
 2   zone                            29991 non-null  object  
 3   occupation                      29991 non-null  object  
 4   income_levels                   29991 non-null  object  
 5   consume_frequency(weekly)       29991 non-null  object  
 6   current_brand                   29991 non-null  object  
 7   preferable_consumption_size     29991 non-null  object  
 8   awareness_of_other_brands       29991 non-null  object  
 9   reasons_for_choosing_brands     29991 non-null  object  
 10  flavor_preference               29991 non-null  object  
 11  purchase_channel                29991 non-null  object  
 12  packaging_preferen

In [92]:
survey_df_3['cf_ab_score'] = survey_df_3['frequency_score'] / (survey_df_3['awareness_score'] + survey_df_3['frequency_score'])

survey_df_3['cf_ab_score'] = survey_df_3['cf_ab_score'].round(2)

survey_df_3 = survey_df_3.drop(columns=['frequency_score', 'awareness_score'])



In [93]:
survey_df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29991 entries, 0 to 29990
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   respondent_id                   29991 non-null  object  
 1   gender                          29991 non-null  object  
 2   zone                            29991 non-null  object  
 3   occupation                      29991 non-null  object  
 4   income_levels                   29991 non-null  object  
 5   consume_frequency(weekly)       29991 non-null  object  
 6   current_brand                   29991 non-null  object  
 7   preferable_consumption_size     29991 non-null  object  
 8   awareness_of_other_brands       29991 non-null  object  
 9   reasons_for_choosing_brands     29991 non-null  object  
 10  flavor_preference               29991 non-null  object  
 11  purchase_channel                29991 non-null  object  
 12  packaging_preferen

In [116]:
survey_df_3.cf_ab_score.max()

0.75

What is the maximum value in cf_ab_score (Consume Frequency and Awareness Brand Score)?
# Ans-1:0.75

# Step 3: Create Zone Affluence Score (ZAS)
 Objective: Calculate the `zas_score` using a combination of the `zone` and
 `income_levels` columns.
 - Usethefollowing mappings for the zone column:
     - "Urban" → 3
     - "Metro" → 4
     - "Rural" → 1
     - "Semi-Urban" → 2
 - Usethefollowing mappings for the income_levels column:
     - "<10L" →1
     - "10L- 15L" →2
     - "16L- 25L" →3
     - "26L- 35L" →4
     - ">35L" →5
     - "Not Reported" → 0
 - Calculate the zas_score using the following formula:
     $$
\text{zas\_score} = \text{zone\_score}*{\text{income\_score}}
$$

In [94]:
survey_df_4=survey_df_3.copy()

In [95]:
survey_df_4.zone.unique()

array(['Urban', 'Metro', 'Rural', 'Semi-Urban'], dtype=object)

In [96]:
survey_df_4.income_levels.unique()

array(['<10L', '> 35L', '16L - 25L', 'Not Reported', '10L - 15L',
       '26L - 35L'], dtype=object)

In [97]:

zone_mapping = {
    "Urban": 3,
    "Metro": 4,
    "Rural": 1,
    "Semi-Urban": 2
}

income_mapping = {
    "<10L": 1,
    "10L - 15L": 2,
    "16L - 25L": 3,
    "26L - 35L": 4,
    "> 35L": 5,
    "Not Reported": 0
}

survey_df_4['zone_score'] = survey_df_4['zone'].map(zone_mapping)
survey_df_4['income_score'] = survey_df_4['income_levels'].map(income_mapping)




In [98]:
survey_df_4.isnull().sum()

respondent_id                     0
gender                            0
zone                              0
occupation                        0
income_levels                     0
consume_frequency(weekly)         0
current_brand                     0
preferable_consumption_size       0
awareness_of_other_brands         0
reasons_for_choosing_brands       0
flavor_preference                 0
purchase_channel                  0
packaging_preference              0
health_concerns                   0
typical_consumption_situations    0
price_range                       0
age_group                         0
cf_ab_score                       0
zone_score                        0
income_score                      0
dtype: int64

In [99]:


survey_df_4['zas_score'] = survey_df_4['zone_score'] * survey_df_4['income_score']


survey_df_4 = survey_df_4.drop(columns=['zone_score', 'income_score'])

In [100]:
survey_df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29991 entries, 0 to 29990
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   respondent_id                   29991 non-null  object  
 1   gender                          29991 non-null  object  
 2   zone                            29991 non-null  object  
 3   occupation                      29991 non-null  object  
 4   income_levels                   29991 non-null  object  
 5   consume_frequency(weekly)       29991 non-null  object  
 6   current_brand                   29991 non-null  object  
 7   preferable_consumption_size     29991 non-null  object  
 8   awareness_of_other_brands       29991 non-null  object  
 9   reasons_for_choosing_brands     29991 non-null  object  
 10  flavor_preference               29991 non-null  object  
 11  purchase_channel                29991 non-null  object  
 12  packaging_preferen

In [119]:
survey_df_4.zas_score.unique().shape

(14,)

How many unique values are present in the Zone Affluence Score (ZAS)?
# Ans-2:14

# Step 4: Brand Switching Indicator (BSI)
 Objective: Create a binary indicator column `bsi` that identifies if a respondent is
 likely to switch brands.
- Check if the respondent’s current_brand is not "Established".
- Also check if the reasons_for_choosing_brands are either "Price" or
 "Quality".
- If both conditions are true, assign a value of 1 to indicate potential for brand
 switching. Otherwise, assign 0.

In [101]:
survey_df_4['reasons_for_choosing_brands'].unique()

array(['Price', 'Quality', 'Availability', 'Brand Reputation'],
      dtype=object)

In [102]:
survey_df_4['current_brand'].unique()

array(['Newcomer', 'Established'], dtype=object)

In [103]:
survey_df_5=survey_df_4.copy()

In [104]:

condition1 = survey_df_5['current_brand'] != "Established"  # Check if current_brand is not "Established"
condition2 = survey_df_5['reasons_for_choosing_brands'].isin(['Price', 'Quality'])  # Check if reasons are "Price" or "Quality"


survey_df_5['bsi'] = (condition1 & condition2).astype(int)  # Assign 1 if both conditions are true, else 0




In [105]:
survey_df_5['bsi']

0        1
1        0
2        0
3        0
4        0
        ..
29986    0
29987    0
29988    0
29989    0
29990    1
Name: bsi, Length: 29991, dtype: int32

# Final Cleaning Step:
 Removing Logical Outliers:
 - When reviewing the occupation data, we found logical inconsistencies.
   
For instance, there are students listed in the `56-70` age group, which seems like an
 incorrect entry.
 
 We need to remove such records where the data doesn't make
 sense logically.

In [106]:
survey_df_5[survey_df_5['occupation']=='Student']['age_group'].unique()

['18-25', '26-35', '56-70']
Categories (6, object): ['18-25' < '26-35' < '36-45' < '46-55' < '56-70' < '70+']

In [107]:
survey_df_5[survey_df_5['occupation']=='Retired']['age_group'].unique()

['56-70']
Categories (6, object): ['18-25' < '26-35' < '36-45' < '46-55' < '56-70' < '70+']

In [108]:
# so only handle student 56-70

In [109]:
survey_df_5[(survey_df_5['occupation']=='Student') & (survey_df_5['age_group']=='56-70')].shape

(35, 20)

In [110]:
# AS only 35 records , lets remove them

In [111]:
survey_df_5.shape

(29991, 20)

In [112]:
survey_df_5[((survey_df_5['occupation']!='Student') | (survey_df_5['age_group']!='56-70'))].shape

(29956, 20)

In [113]:
survey_df_6 = survey_df_5[~((survey_df_5['occupation']=='Student') & (survey_df_5['age_group']=='56-70'))]

In [114]:
survey_df_6.shape

(29956, 20)

How many rows are present in the dataset after removing logical outliers in the Age Group column?
# Ans-3:29956

In [122]:
survey_df_6[survey_df_6.bsi==0].shape

(20796, 20)

How many rows contain the Brand Switching Indicator (BSI) with a value of 0 after removing logical outliers?
# Ans-4:

# Saving file

In [115]:
survey_df_6.to_csv('survey_data_feature_engineered.csv',index=False)